In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
#this code
import os
import librosa
import numpy as np

    # Load audio
def generate_mel_spectrogram(audio_path, annotation_path, output_dir, sr=22050, n_fft=2048, hop_length=512, n_mels=128):
    y, sr = librosa.load(audio_path, sr=sr)

    # Read annotations
    with open(annotation_path, 'r') as f:
        annotations = f.readlines()

    # Process each annotation with label "song" or "call"
    for annotation in annotations:
        start_sec, end_sec, label = annotation.split()
        start_sec, end_sec = map(float, (start_sec, end_sec))
        if label in ["song", "call", "drumming", "clapping"]:
            # Iterate through each second within the annotated range
            for sec in np.arange(start_sec, end_sec + 1):
                start_frame = int(sec * sr)
                end_frame = int((sec + 1) * sr)

                # Check if segment length is long enough for n_fft
                if end_frame - start_frame >= n_fft:
                    # Extract segment
                    segment = y[start_frame:end_frame]

                    # Compute mel spectrogram
                    S = librosa.feature.melspectrogram(y=segment, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
                    S_db = librosa.power_to_db(S, ref=np.max)

                    # Save spectrogram as .npy
                    output_filename = f"{label}_{sec}.npy"
                    np.save(os.path.join(output_dir, output_filename), S_db)

def process_dataset(dataset_dir, output_dir):
    for root, dirs, files in os.walk(dataset_dir):
        for file in files:
            if file.endswith('.mp3'):
                audio_path = os.path.join(root, file)
                annotation_path = os.path.join(root, file.replace('.mp3', '.txt'))
                output_subdir = os.path.join(output_dir, os.path.basename(root))
                os.makedirs(output_subdir, exist_ok=True)
                generate_mel_spectrogram(audio_path, annotation_path, output_subdir)

# Example usage
dataset_dir = '/content/drive/My Drive/birdaudio/'
output_dir = '/content/drive/My Drive/spectrogramnpy_main/'
process_dataset(dataset_dir, output_dir)